In [ ]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
# os.environ['JAVA_HOME']='/usr'
# os.environ['SPARK_HOME'] ='/usr/local/lib/python3.9/site-packages/pyspark/'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as F
# from pyspark.sql.functions import lpad, rpad

In [ ]:
conf = SparkConf().setAppName('ShowWithCollect')\
    .setMaster('yarn') \
    .set('spark.default.parallelism', '4')

spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()

sc = spark.sparkContext
print("app_id".ljust(50), sc.applicationId)

In [ ]:
import random
import pyspark.sql.functions as F
from datetime import date, timedelta

end = date.today()
start = end + timedelta(days=-365)
length_customers = 100

names = ['Bob', 'James', 'Marek', 'Johannes', 'Pierre', 'Jean', 'Thomas']
countries = ['bel', 'usa', 'fra', 'aus', 'ger', 'pol']

In [ ]:
customers = list()
for i in range(length_customers):
    random_date = start + (end - start) * random.random()
    customer = (names[random.randint(0, len(names)-1)],         # name
                countries[random.randint(0, len(countries)-1)], # country
                random_date,                                    # last_order_date
                random.randint(-10000, 10000)               # balance
                )                                    
    customers.append(customer)

In [ ]:
dfCust = spark.createDataFrame(data=customers, schema=["name","country","last_ord_dt", "bal"])
collected = dfCust.collect()
print(type(collected))

([' | '.join([i.name.ljust(12, ' '), 
              i.country, 
              str(i.last_ord_dt), 
              str(i.bal).rjust(10, ' ')]) for i in collected])

In [ ]:
# df_customers = df_customers.withColumn("name", F.rpad(F.col("name"), 10, " "))\
#     .withColumn("last_ord_dt", F.date_format(F.col("last_ord_dt"),"yyyy-MM-dd"))\
#     .withColumn("bal_str", F.format_number(F.col("bal").cast("double"), 2))\
#     .withColumn("bal", F.lpad(F.col("bal_str"), 12, " "))\
#     .drop("bal_str")

In [ ]:
df_customers = df_customers.withColumn("name", F.rpad(F.col("name"), 10, " "))\
    .withColumn("last_ord_dt", F.date_format(F.col("last_ord_dt"),"yyyy-MM-dd"))\
    .withColumn("bal_str", F.format_number(F.col("bal").cast("double"), 2))\
    .withColumn("bal_str", F.regexp_replace(F.regexp_replace(F.regexp_replace(
            F.col("bal_str"), '\\.', '#'), ',', '\\.'), '#', ','))\
    .withColumn("bal", F.lpad(F.col("bal_str"), 14, " "))\
    .drop("bal_str")
    # .withColumn("bal_string", F.lpad(df_customers["bal_string"], 8, " "))\


    # .withColumn("bal", F.lpad(df_customers["bal"], 8, " "))\
# .withColumn("russian_format", F.regexp_replace(F.regexp_replace(F.regexp_replace(
#         F.col("bal_string"), '\\.', '#'), ',', ' '), '#', ','))\

In [ ]:
--

In [ ]:
spark.stop()